## updated chip seq notebook: 
- All of the datasets downloaded in previous notebook lacked information
- I think that the excel files have information I need, which means I need to redo all of the work from the other notebook with downloaded excel files as pandas dataframes
- 166 chipseq d. mel datasets from modENCODE
- Only downloading 159 of those because modMINE lacks information for the rest (see no_data list)

In [1]:
import pandas as pd
import urllib
import os
from urllib.request import urlopen

### Download from modENCODE: 

In [ ]:
url = 'http://data.modencode.org/cgi-bin/cloud_list.pl?accessions=3393,3806,3825,3231,2625,2626,2637,3403,4078,3240,4080,5068,4082,4081,3959,5069,2638,2639,3395,3235,4974,5008,5070,5071,5072,5577,3229,3230,3402,3401,2640,2641,2642,3234,3236,3239,3241,3400,3398,4976,3824,3826,4089,3809,3238,3397,5028,3814,3245,3830,4119,4981,5257,3815,4982,3827,2629,2630,2635,2636,2633,2634,2627,2628,2631,2632,3391,3392,4091,3956,4936,5592,5111,5110,5590,5587,5112,5591,5264,5113,5593,4944,3390,5023,4998,4070,3242,5114,5575,5115,5594,3243,3394,3812,5004,5005,5116,5597,5574,5117,4094,5118,854,858,3233,862,863,856,849,857,855,985,859,860,861,5119,4074,4107,3957,3237,3813,3816,5120,5121,5598,4095,4096,3960,3808,4098,5576,5570,5579,5571,984,850,848,4352,5580,5569,5122,5123,5124,846,847,851,852,853,5125,5126,5599,4099,4071,3232,5568,5014,4103,5017,4069,4104,3958,4716,4105,4943,5127,5606,3396,5128,5024,5025,3399,4114,4113,3823,5029,5129&urls=1'
#protein=',3954,4351,2755,3253,2753,2979,2754,3625,2783,3247,3251,4193'
page = urlopen(url).read().decode('utf-8')
page = page.split('\n')

modid_list = []
for line in page:
    if not line.startswith('#'):
        modid = str(line.split()[0]) #gives an error but still works? 
        modid_list.append(modid)

In [ ]:
len(set(modid_list))

In [ ]:
#define 18? ids that modmine has no data for (unknown reasons...)
no_data = ['2635', '2625', '2637', '2631', '3806', '2632', '3827', '3826', '3235', '3236', '3823', '3824', '3825', '2640', '3231', '3397', '3233', '2628'] 

In [ ]:
#Note: Modmine sometimes goes down unexpectedly
PATH = '../../data/chip/modENCODE_downloads/'
DWNLD1 = 'http://intermine.modencode.org/release-33/features.do?type=submission&action=export&format=csv&submission=modENCODE_'
DWNLD2 = '&feature=BindingSite'
os.makedirs(PATH, exist_ok=True)

done = []
for name in modid_list: 
    if name not in no_data: 
        if name not in done: 
            #try:
            urllib.request.urlretrieve(DWNLD1+name+DWNLD2, PATH+name) 
            done.append(name)
            #except:
                #urllib.request.urlretrieve(DWNLD1+name+'&feature=ProteinBindingSite', PATH+name) 
                #done.append(name)

After the download:
- combine files to make one big file
- get as bed file
- do liftover
- intersect w gene bed file (one idea: if it's too hard to get these as beds, could use data from before and merge on location) 
- clean up table
- phantom peak overlap? 

In [ ]:
#combine files into one: 
concat=[]
import glob
for fname in glob.glob('../../data/modENCODE_downloads_2ndtry/[0-9]*'):
    name = os.path.splitext(os.path.basename(fname))[0]
    if name not in no_data:
        df = pd.read_csv(fname)
        #df.astype(int)
        #df[2]
        concat.append(df)     

In [ ]:
big = pd.concat(concat)
big.to_csv('../../data/modENCODE_downloads_2ndtry/modENCODE_big', sep='\t', na_rep=':.', header=None, index=False)

### Cleanup of big table to prep for snakemake steps: 

- trying to keep everything in the same structure from now on

In [48]:
big = pd.read_table('../../output/chip/modENCODE_downloads_2ndtry/modENCODE_big', header=None)

/Users/bergeric/miniconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [49]:
#make new id column (set to name/4th column)
#make this a bed and add chr to chromosome 
big.columns = ['DB_id','score','chrom','start','end','strand','modENCODE_id','type','name']

In [50]:
big['id'] = ['id='+str(x) for x in range(0, len(big.score))]

In [51]:
big['chrom'] = ['chr'+x for x in big['chrom']]

In [52]:
bigbed = big[['chrom','start','end','id','score','strand','modENCODE_id','type','name','DB_id']]

In [53]:
bigbed.loc[bigbed.score == ':.', 'score'] = '.'

In [55]:
#another = another.fillna('.')
happy = bigbed.set_index(
    ['chrom','start','end','id','score','strand','modENCODE_id','type','DB_id']).pivot_table(
        columns='type', index=['chrom','start','end','id','score','strand','modENCODE_id','DB_id'], 
        values='name',aggfunc='first')
happy = happy.reset_index()

In [58]:
happy.head()

type,chrom,start,end,id,score,strand,modENCODE_id,DB_id,antibody,cell line,developmental stage,strain,target gene,threshold,tissue
0,chr2L,16,6825,id=307512,1.80011,0,modENCODE_3956,Solexa_HP1_wa191_14_16_hr_OR_embryo.gff_ID015267,None,None,None,Oregon-R-modENCODE,None,None,None
1,chr2L,16,6825,id=307513,1.80011,0,modENCODE_3956,Solexa_HP1_wa191_14_16_hr_OR_embryo.gff_ID015267,None,None,Embryo 14-16hr OR,None,None,None,None
2,chr2L,16,6825,id=307514,1.80011,0,modENCODE_3956,Solexa_HP1_wa191_14_16_hr_OR_embryo.gff_ID015267,HP1 wa191,None,None,None,None,None,None
3,chr2L,19,6870,id=307515,1.85852,0,modENCODE_3956,Solexa_HP1_wa191_14_16_hr_OR_embryo.enriched_r...,None,None,None,Oregon-R-modENCODE,None,None,None
4,chr2L,19,6870,id=307516,1.85852,0,modENCODE_3956,Solexa_HP1_wa191_14_16_hr_OR_embryo.enriched_r...,None,None,Embryo 14-16hr OR,None,None,None,None


In [40]:
#parse before GFP from DB_id: 
hopefully_genes = []
for item in bigbed.DB_id:
    new = item.split('_GFP')[0]
    new = new.split('.gff')[0]
    new = new.split('.')[0]
    hopefully_genes.append(new)

In [41]:
len(set(hopefully_genes))

148

In [42]:
# relates modENCODE_id to TF targeted: 
# The following two lines will be needed in every python script:
from intermine.webservice import Service
service = Service("http://intermine.modencode.org/release-33/service")

# Get a new query on the class (table) you will be querying:
query = service.new_query("Submission")

# The view specifies the output columns
query.add_view("DCCid", "assayFactor")

# Uncomment and edit the line below (the default) to select a custom sort order:
# query.add_sort_order("Submission.DCCid", "ASC")

# You can edit the constraint values below
query.add_constraint("experimentType", "=", "ChIP-seq", code = "A")
query.add_constraint("organism.commonName", "=", "fruit fly", code = "B")
#query.add_constraint("lab.name", "=", "Kevin White", code = "C")

# Uncomment and edit the code below to specify your own custom logic:
# query.set_logic("A and B and C")
table = []
for row in query.rows():
    newline = [row["DCCid"], row["assayFactor"]]
    table.append(newline)

In [62]:
gene_map = pd.DataFrame(table)
gene_map.columns = ['modENCODE_id','SYMBOL']
gene_map.head()

,modENCODE_id,SYMBOL
0,modENCODE_2626,Caudal
1,modENCODE_2627,HDAC4a
2,modENCODE_2629,HDAC11
3,modENCODE_2630,HDAC11
4,modENCODE_2633,HDAC3


In [73]:
withmap = happy.merge(gene_map, on='modENCODE_id', how='left')

In [74]:
#add column for motif_FBgn
symbolmap = pd.read_table('/data/LCDB/lcdb-references/dmel/r6-11/gtf/dmel_r6-11.SYMBOL.csv', sep=',') 
addfbgn = withmap.merge(symbolmap, on='SYMBOL', how='left')
addfbgn = addfbgn.rename(columns={'ENSEMBL': 'TF_FBgn'})

In [75]:
addfbgn = addfbgn.rename(columns={'SYMBOL': 'TF'})

In [76]:
addfbgn.head()

,chrom,start,end,id,score,strand,modENCODE_id,DB_id,antibody,cell line,developmental stage,strain,target gene,threshold,tissue,TF,motif_FBgn
0,chr2L,16,6825,id=307512,1.80011,0,modENCODE_3956,Solexa_HP1_wa191_14_16_hr_OR_embryo.gff_ID015267,None,None,None,Oregon-R-modENCODE,None,None,None,HP1a,NaN
1,chr2L,16,6825,id=307513,1.80011,0,modENCODE_3956,Solexa_HP1_wa191_14_16_hr_OR_embryo.gff_ID015267,None,None,Embryo 14-16hr OR,None,None,None,None,HP1a,NaN
2,chr2L,16,6825,id=307514,1.80011,0,modENCODE_3956,Solexa_HP1_wa191_14_16_hr_OR_embryo.gff_ID015267,HP1 wa191,None,None,None,None,None,None,HP1a,NaN
3,chr2L,19,6870,id=307515,1.85852,0,modENCODE_3956,Solexa_HP1_wa191_14_16_hr_OR_embryo.enriched_r...,None,None,None,Oregon-R-modENCODE,None,None,None,HP1a,NaN
4,chr2L,19,6870,id=307516,1.85852,0,modENCODE_3956,Solexa_HP1_wa191_14_16_hr_OR_embryo.enriched_r...,None,None,Embryo 14-16hr OR,None,None,None,None,HP1a,NaN


In [65]:
symbolmap2 = pd.read_table('../../data/fb_synonym.tsv', sep=' ', header=None)
symbolmap2.columns = ['gene','a','b']
symbolmap2.head()

,gene,a,b
0,FBgn0262029,d,dachs
1,FBgn0052532,CG32532,"CG14203,CG14202,CG14201"
2,FBgn0023536,CG3156,EG:171D11.2
3,FBgn0029718,mRpL30,mitochondrial
4,FBgn0031101,CG1631,NaN


In [77]:
for val in set(addfbgn.TF):
    if val not in list(symbolmap2.a):
        if val not in list(symbolmap2.b):
            print(val)

dm
Yki
KNI
POF
Smc3
Su(Hw)
RNA polymerase II
PCL
Caudal
DLL
CP190
HP2
Huckebein
HDAC4a
JHDM1
MOF
GROUCHO
HP1a
LSD1
ZW5
RPD3
GAF
dMi-2
eGFP
RNA polymerase II CTD repeat YSPTSPS (phospho S2)
Chriz
RNA polymerase II CTD repeat YSPTSPS (phospho S5)
NURF301
CG16778
dRING
JMJD2A/KDM4A
dSFMBT


Change all of the incorrect gene names so that symbolmap2 can recognize them: 

In [78]:
addfbgn_copy = addfbgn.copy()

In [79]:
addfbgn_copy.loc[addfbgn.TF == 'dMi-2','TF'] = 'Mi-2'
addfbgn_copy.loc[addfbgn.TF == 'Smc3','TF'] = 'SMC3'
addfbgn_copy.loc[addfbgn.TF == 'Caudal','TF'] = 'caudal'
addfbgn_copy.loc[addfbgn.TF == 'MOF','TF'] = 'mof'
addfbgn_copy.loc[addfbgn.TF == 'JMJD2A/KDM4A','TF'] = 'Kdm4A'
addfbgn_copy.loc[addfbgn.TF == 'dRING','TF'] = 'Sce'
addfbgn_copy.loc[addfbgn.TF == 'Chriz','TF'] = 'Chro'
addfbgn_copy.loc[addfbgn.TF == 'HP1a','TF'] = 'Su(var)205'
addfbgn_copy.loc[addfbgn.TF == 'LSD1','TF'] = 'Su(var)3-3'
addfbgn_copy.loc[addfbgn.TF == 'POF','TF'] = 'Pof'
addfbgn_copy.loc[addfbgn.TF == 'DLL','TF'] = 'Dll'
addfbgn_copy.loc[addfbgn.TF == 'CP190','TF'] = 'Cp190'
addfbgn_copy.loc[addfbgn.TF == 'GROUCHO','TF'] = 'groucho'
addfbgn_copy.loc[addfbgn.TF == 'Yki','TF'] = 'yki'
addfbgn_copy.loc[addfbgn.TF == 'GAF','TF'] = 'Trl'
addfbgn_copy.loc[addfbgn.TF == 'RPD3','TF'] = 'HDAC1'
addfbgn_copy.loc[addfbgn.TF == 'HP2','TF'] = 'Su(var)2-HP2'
addfbgn_copy.loc[addfbgn.TF == 'Su(Hw)','TF'] = 'su(Hw)'
addfbgn_copy.loc[addfbgn.TF == 'NURF301','TF'] = 'E(bx)'
addfbgn_copy.loc[addfbgn.TF == 'dSFMBT','TF'] = 'Sfmbt'
addfbgn_copy.loc[addfbgn.TF == 'ZW5','TF'] = 'dwg'
addfbgn_copy.loc[addfbgn.TF == 'JHDM1','TF'] = 'Kdm2'
addfbgn_copy.loc[addfbgn.TF == 'CG16778','TF'] = 'lov'
addfbgn_copy.loc[addfbgn.TF == 'dm','TF'] = 'Myc'
addfbgn_copy.loc[addfbgn.TF == 'KNI','TF'] = 'kni'
addfbgn_copy.loc[addfbgn.TF == 'PCL','TF'] = 'Pcl'
addfbgn_copy.loc[addfbgn.TF == 'Huckebein','TF'] = 'hkb'
addfbgn_copy.loc[addfbgn.TF == 'HDAC4a','TF'] = 'HDAC4'

In [80]:
for val in set(addfbgn_copy.TF):
    if val not in list(symbolmap2.a):
        if val not in list(symbolmap2.b):
            print(val)

eGFP
RNA polymerase II CTD repeat YSPTSPS (phospho S2)
RNA polymerase II CTD repeat YSPTSPS (phospho S5)
RNA polymerase II


In [81]:
merge2 = addfbgn_copy.merge(symbolmap2, left_on='TF', right_on='a', how='left').merge(symbolmap2, left_on='TF', right_on='b', how='left')

In [86]:
merge2.fillna('')
merge2['motif_FBgn_sum']= merge2.TF_FBgn.combine_first(merge2.gene_x).combine_first(merge2.gene_y)

In [87]:
merge2.columns

Index(['chrom', 'start', 'end', 'id', 'score', 'strand', 'modENCODE_id',
       'DB_id', 'antibody', 'cell line', 'developmental stage', 'strain',
       'target gene', 'threshold', 'tissue', 'TF', 'motif_FBgn', 'gene_x',
       'a_x', 'b_x', 'gene_y', 'a_y', 'b_y', 'motif_FBgn_sum'],
      dtype='object')

In [88]:
almost = merge2[['chrom', 'start', 'end', 'id', 'score', 'strand', 'modENCODE_id',
       'DB_id', 'antibody', 'cell line', 'developmental stage', 'strain',
       'target gene', 'threshold', 'tissue', 'TF', 'motif_FBgn_sum']]

In [89]:
len(almost.motif_FBgn_sum.unique())

72

In [90]:
almost = almost.rename(columns={'motif_FBgn_sum': 'TF_fbgn'})

In [91]:
almost.head()

,chrom,start,end,id,score,strand,modENCODE_id,DB_id,antibody,cell line,developmental stage,strain,target gene,threshold,tissue,TF,TF_fbgn
0,chr2L,16,6825,id=307512,1.80011,0,modENCODE_3956,Solexa_HP1_wa191_14_16_hr_OR_embryo.gff_ID015267,None,None,None,Oregon-R-modENCODE,None,None,None,Su(var)205,FBgn0003607
1,chr2L,16,6825,id=307513,1.80011,0,modENCODE_3956,Solexa_HP1_wa191_14_16_hr_OR_embryo.gff_ID015267,None,None,Embryo 14-16hr OR,None,None,None,None,Su(var)205,FBgn0003607
2,chr2L,16,6825,id=307514,1.80011,0,modENCODE_3956,Solexa_HP1_wa191_14_16_hr_OR_embryo.gff_ID015267,HP1 wa191,None,None,None,None,None,None,Su(var)205,FBgn0003607
3,chr2L,19,6870,id=307515,1.85852,0,modENCODE_3956,Solexa_HP1_wa191_14_16_hr_OR_embryo.enriched_r...,None,None,None,Oregon-R-modENCODE,None,None,None,Su(var)205,FBgn0003607
4,chr2L,19,6870,id=307516,1.85852,0,modENCODE_3956,Solexa_HP1_wa191_14_16_hr_OR_embryo.enriched_r...,None,None,Embryo 14-16hr OR,None,None,None,None,Su(var)205,FBgn0003607


In [22]:
almost.to_csv('../../output/chip/modENCODE_big.bed', sep='\t', index=False)

## snakemake: 

- cut table
- liftover
- get gene file in bed format
- intersect
- merge metadata back

In [24]:
#get gene file as a bed:
import pybedtools
from pybedtools import BedTool
from pybedtools.featurefuncs import gff2bed
genes = BedTool('../../data/dmel-all-r6.12.gene_only.chr.gff')
genes_bed = genes.each(gff2bed, name_field='ID').saveas()

In [26]:
#import liftover file, bed intersect
with open('../../output/chip/modENCODE_big_liftover.bed') as f: 
    peaks_bed = BedTool(f)
    intersect = genes_bed.intersect(peaks_bed, wb=True).saveas()
    intdf = intersect.to_dataframe()
#then merge back on location to get all the info

In [27]:
trim = intdf[['name','thickStart','thickEnd','itemRgb','blockCount']]

In [35]:
trim.columns = [['gene', 'chrom','start','end','id']]

In [36]:
#in snakemake, instead of merging big I merge just the metadata (a lot cleaner)
metadata = bigbed[['id','score', 'strand', 'modENCODE_id','type', 'name', 'DB_id']]
merge = trim.merge(metadata, on='id', how='left')
merge.head()

,gene,chrom,start,end,id,score,strand,modENCODE_id,type,name,DB_id
0,FBgn0031208,chr2L,7492,8128,id=280047,51.15,0,modENCODE_3830,strain,yellow cinnabar brown speck,Trithorax_like_WPP_ChIP_chip.gff_ID000002
1,FBgn0031208,chr2L,7492,8128,id=280048,51.15,0,modENCODE_3830,developmental stage,L3 stage larvae,Trithorax_like_WPP_ChIP_chip.gff_ID000002
2,FBgn0031208,chr2L,7492,8128,id=280049,51.15,0,modENCODE_3830,antibody,KW3-Trl-D2,Trithorax_like_WPP_ChIP_chip.gff_ID000002
3,FBgn0031208,chr2L,5038,13342,id=383688,1.9869,0,modENCODE_3957,strain,Oregon-R-modENCODE,Solexa_Pc_14_16_hr_OR_embryo.gff_ID000002
4,FBgn0031208,chr2L,5038,13342,id=383689,1.9869,0,modENCODE_3957,developmental stage,Embryo 14-16hr OR,Solexa_Pc_14_16_hr_OR_embryo.gff_ID000002


In [37]:
len(merge.modENCODE_id.unique())

150

In [ ]:
#this is for peak_location_fbgn (gene) which I don't have yet
almost = almost.rename(columns={'gene_x': 'peak_location_fbgn'})

In [ ]:
#merge to add column for peak_location_symbol
almost1 = almost.merge(symbolmap, left_on='peak_location_fbgn', right_on='ENSEMBL', how='left')
almost1 = almost1.rename(columns={'SYMBOL': 'peak_location_symbol'})
almost1.columns
final = almost1[['DB_id', 'score', 'chrom', 'start', 'end', 'strand', 'modENCODE_id',
       'peak_location_fbgn',
       'peak_location_symbol', 'antibody', 'cell line', 'developmental stage',
       'strain', 'target gene','tissue', 'TF', 'TF_fbgn']]
    

In [ ]:
final.to_csv('../../data/modENCODE_downloads_2ndtry/modENCODE_finaltable', sep='\t', header=None, index=False)

In [ ]:
final.tissue.unique()

In [ ]:
len(final['developmental stage'].unique())

In [ ]:
final['cell line'].unique()

In [ ]:
len(final.strain.unique())

In [ ]:
final[['developmental stage', 'modENCODE_id']].drop_duplicates().groupby('developmental stage').describe()

In [ ]:
final[['cell line', 'modENCODE_id']].drop_duplicates().groupby('cell line').describe()